# Setup

In [7]:
import numpy as np
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier
from scipy.stats import randint
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report, auc
import scikitplot as skplt  # search for scikit-plot
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna
import xgboost as xgb

# Load the training data and the test inputs
x_train = pd.read_csv('X_train.csv', index_col=0, header=[0, 1, 2])
x_train_np = np.array(x_train)
y_train = pd.read_csv('y_train.csv', index_col=0)
y_train_np = y_train.squeeze().to_numpy()  # Make y_train a NumPy array
x_test = pd.read_csv('X_test.csv', index_col=0, header=[0, 1, 2])
x_test_np = np.array(x_test)

x_train_flat_columns = ['_'.join(col).strip() for col in x_train.columns.values]
x_train.columns = x_train_flat_columns

x_test_flat_columns = ['_'.join(col).strip() for col in x_test.columns.values]
x_test.columns = x_train_flat_columns
# Prepare data
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_np.ravel())

# Split training data into training and temporary validation sets
X_train, X_temp, Y_train, Y_temp = train_test_split(x_train, y_train_encoded, test_size=0.4, random_state=42)

# Split the temporary validation set into validation and fake test set
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)
X_real_test_scaled = scaler.transform(x_test)  # real test set we don't have labels for


def calculate_test_accuracy(predictions):
    return f"Pseudo Test Set accuracy: {accuracy_score(Y_test, predictions):.2f}"


def calculate_trainig_accuracy(predictions):
    return f"Training Set accuracy: {accuracy_score(Y_train, predictions):.2f}"


def split_features_by_type(X, feature_structure):
    """
    Splits the dataset into subsets based on the feature structure provided.

    :param X: numpy array, the dataset to be split (features only)
    :param feature_structure: dict, keys are feature names and values are the number of features of that type
    :return: dict of feature subsets
    """
    feature_subsets = {}
    start_idx = 0
    
    for feature_name, feature_count in feature_structure.items():
        end_idx = start_idx + feature_count
        feature_subsets[feature_name] = X[:, start_idx:end_idx]
        start_idx = end_idx
    
    return feature_subsets

# Define the structure of your features based on the information you've provided
feature_structure = {
    'chroma_cens': 84,
    'chroma_cqt': 84,
    'chroma_stft': 84,
    'mfcc': 140,
    'rmse': 7,
    'spectral_bandwidth': 7,
    'spectral_centroid': 7,
    'spectral_contrast': 49,
    'spectral_rolloff': 7,
    'tonnetz': 42,
    'zcr': 7
}

# Example usage with a hypothetical dataset X_train_scaled
# This would be your preprocessed and scaled training data as a NumPy array
train_feature_subsets = split_features_by_type(X_train_scaled, feature_structure)
val_feature_subsets = split_features_by_type(X_val_scaled, feature_structure)
test_feature_subsets = split_features_by_type(X_test_scaled, feature_structure)

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier

# Number of base estimators
n_estimators = 10  # You can experiment with this number

# Create a BaggingClassifier with KNN as base estimators
bagging_clf = BaggingClassifier(
    estimator=KNeighborsClassifier(n_neighbors=1), 
    n_estimators=n_estimators, 
    max_samples=1.0 / n_estimators, 
    bootstrap=True, 
    random_state=42,
    n_jobs=-1  # Use all cores
)

# Fit the BaggingClassifier
bagging_clf.fit(X_train_scaled, Y_train)
# training predictions
trainpreds = bagging_clf.predict(X_train_scaled)
# test predictions
testpreds = bagging_clf.predict(X_test_scaled)

print(calculate_trainig_accuracy(trainpreds))
print(calculate_test_accuracy(testpreds))

Training Set accuracy: 0.52
Pseudo Test Set accuracy: 0.40


### Function to save EPS Plots directly to Overleaf

In [ ]:
import getpass
import os
export_username = "ts" # Only save plots to dropbox on Tobias's code
def save_plot(plot, filename):
    username = getpass.getuser()
    filepath = "/Users/ts/Library/CloudStorage/Dropbox/Apps/Overleaf/SML Practical/Figures"
    filename += ".eps"
    if username == export_username:
        plot.savefig(os.path.join(filepath, filename), format='eps') # Save as EPS

# EDA

## Check Balance of Classes

In [ ]:
class_bal = plt.figure(figsize=(10, 6))
sns.countplot(data=y_train, y='Genre')
plt.title('Distribution of Genres in the Training Set')
plt.xlabel('Count')
plt.ylabel('Genre')
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
plt.show()
save_plot(class_bal, "Class-Balance")

## Plot Distribution of Medians across feature subset

In [ ]:
x_train_with_genre = x_train.merge(y_train, left_index=True, right_on='Id') # Merge Genre labels on to training data
# Create the subplots
box1, axs = plt.subplots(nrows=2, ncols=2, figsize=(16, 9))

# Top-left subplot for spectral_centroid_median_01
sns.boxplot(x='spectral_centroid_median_01', y='Genre', data=x_train_with_genre, ax=axs[0, 0])
axs[0, 0].set_title('Spectral Centroid Median 01')

# Top-right subplot for spectral_rolloff_median_01
sns.boxplot(x='spectral_rolloff_median_01', y='Genre', data=x_train_with_genre, ax=axs[0, 1])
axs[0, 1].set_title('Spectral Rolloff Median 01')

# Bottom-left subplot for spectral_contrast_median_04
sns.boxplot(x='spectral_contrast_median_04', y='Genre', data=x_train_with_genre, ax=axs[1, 0])
axs[1, 0].set_title('Spectral Contrast Median 04')

# Bottom-right subplot for mfcc_median_01
sns.boxplot(x='mfcc_median_01', y='Genre', data=x_train_with_genre, ax=axs[1, 1])
axs[1, 1].set_title('MFCC Median 01')

# Adjust the font scale for better readability
sns.set(font_scale=1.6)

# Tight layout for better spacing
plt.tight_layout()

# Show the plots
plt.show()
save_plot(box1, "boxplot-1")

# Julia

# Max

# Tobias